<b><u> News classification and analysis project : 1st phase </u></b>

# `Retrieve data from DeccanHeral archives of November 2015`

In [3]:
# importing the libraries

from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import calendar
import datetime
import time

from json2xml import json2xml
from json2xml.utils import readfromjson

In [4]:
def get_archives_of_month_of_year(month,year):
    opts = Options()
    driver = webdriver.Chrome(r"C:\Users\Admin\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe",options=opts)  # set driver and options

    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36")

    url = 'https://www.deccanherald.com/archives'                                             # url 

    driver.get(url)

    driver.find_element_by_css_selector(".flatpickr").click()                                  # click on the calender input box

    req_year = year                                                                            # get the year
    
    if req_year > datetime.datetime.now().year:
        print(f'Wait till the {req_year} arrives')
        driver.close()
        exit(keep_kernel=True)
        
    

    driver.find_element_by_class_name("numInput").send_keys(Keys.CONTROL + Keys.BACK_SPACE)    # clear current year
    driver.find_element_by_class_name("numInput").send_keys(str(req_year))                     # fill the requried year

    mydate = datetime.datetime.now()                                                           # get current month
    curr_month = mydate.strftime("%B")

    req_month = month.title()                                                                   # get require month with appropriate case

    month_dict = {}
    for i in range(1,13):
        month_dict[calendar.month_name[i]] = i

    # iterations required : 
    no_of_clicks = month_dict[curr_month] - month_dict[req_month]                              # this function gets you to the required required month 
    for _ in range(abs(no_of_clicks)):
        time.sleep(0.2)
        if no_of_clicks > 0 : 
            driver.find_element_by_class_name("flatpickr-prev-month").click()                  # clicks performed to get to the previous month
        elif no_of_clicks < 0:
            driver.find_element_by_class_name("flatpickr-next-month").click()                  # clicks perfomed to get to the next month
        else: 
            pass

    date_l, vertical_l, headline_l , author_l , description_l = [],[],[],[],[]                 # define lists
 

    for i in range(len(driver.find_elements_by_xpath(f"//span[contains(@aria-label,'{req_month}')]"))):         # iterate through all days of the required month
        driver.find_elements_by_xpath(f"//span[contains(@aria-label,'{req_month}')]")[i].click()                # click on the date
        driver.find_element_by_class_name("datepicker-submit").click()                                      # press `submit`   

        ######################################################################################################
        
        time.sleep(2)

        window_before = driver.window_handles[0]                                                            # get root tab

        for j in range(len(driver.find_elements_by_xpath("//ul[@class='group notab-snippets notab-snippets--graybg f-left full-width notab-snippets--archive-note']/li//a"))):  # iterate through all the links per day


            about = driver.find_elements_by_xpath("//div[@id='archive_result']/*//a")[j]

            ActionChains(driver).key_down(Keys.CONTROL).click(about).key_up(Keys.CONTROL).perform()        # open the link in new tab

            window_after = driver.window_handles[-1]                                                       

            driver.switch_to.window(window_after)                                                          # switch tab to newly opened tab

            #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%            

            time.sleep(2)

            sauce = driver.page_source                                           

            soup = BeautifulSoup(sauce)

            date_l.append(soup.find('li',attrs={'class':'sanspro-semib text-uppercase crud-items__lists'}).text)  # get date
            
            try:
                description_l.append(soup.find('p',attrs={'style':'display: block;'}).text)  # get description if possible
            except:
                try:
                    X = soup.find_all('div',attrs={'class':'field-item even'})[1]
                    description_l.append(X.findChildren('p')[0])                            # get description if possible
                except:
                    try:
                        description_l.append(soup.find('div',attrs={'align':'justify'}).text)      # get description if possible
                    except:
                        description_l.append('Not Found')                                       # append `not found` to description for the record
            
            headline_l.append(soup.find('h1',attrs={'id':'page-title'}).text)                    # get headline

            X = soup.find('li',attrs={'class':'last'})
            vertical_l.append(X.previous_sibling.text)                                           # get vertical
            
            try:
                author_l.append(soup.find('span',attrs={'class':'sanspro-reg article-author__name'}).text)    # get author
            except:
                author_l.append('not_available')       # append 'not_available' to author if author not found
    

            #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

            driver.close()                                   # close the current driver ( the one with the opened link )

            driver.switch_to.window(window_before)

        ######################################################################################################

        time.sleep(3)
        ActionChains(driver).move_to_element(driver.find_element_by_css_selector(".flatpickr")).click().perform() # click on the calender input again and iterate to next date
        time.sleep(2)
        
    driver.close()                                              # close the root driver 
        
    df = pd.DataFrame(list(zip(date_l,vertical_l,author_l,headline_l,description_l)),columns=['date','vertical','author','headline','description'])   # create a dataframe
    
    return df       # return the dataframe

In [1]:
result_df = get_archives_of_month_of_year('november',2021)                              # get the result of a particular month of a year

In [ ]:
if not os.path.exists('data'):                                                         # create a directory to store the data
    os.makedirs('data')

In [8]:
# saving the data

result_df.to_csv('DeccanHerald_archive_new_scrapes_November_2015.csv',index=False)      # save the result to a csv file

result_df.to_json('DeccanHerald_archive_new_scrapes_November_2015.json')                # save the result to a json file

data = readfromjson("DeccanHerald_archive_new_scrapes_November_2015.json")
df_xml = json2xml.Json2xml(data).to_xml()                                               # convert json to xml

f =  open("data/DeccanHerald_archive_new_scrapes_November_2015.text", "w",encoding='utf-8')
f.write(df_xml)                                                                         # saving the xml formatted file in text extension file
f.close()              

f =  open("data/DeccanHerald_archive_new_scrapes_November_2015.xml", "w",encoding='utf-8')
f.write(df_xml)                                                                         # saving the xml file 
f.close()